<a href="https://colab.research.google.com/github/dinaldoap/jit-sdp-nn/blob/master/notebook/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import random
from scipy.stats import mstats

In [438]:
df = pd.read_csv('https://raw.githubusercontent.com/dinaldoap/jit-sdp-data/master/jenkins.csv')
df.head()

,fix,ns,nd,nf,entropy,la,ld,lt,ndev,age,nuc,exp,rexp,sexp,author_date_unix_timestamp,classification,contains_bug
0,False,7.0,7.0,7.0,2.641604,9.0,9.0,426.428571,100.0,0.000093,1.0,5171.0,30.227271,1472.714286,1555326371,None,False
1,False,7.0,7.0,7.0,2.750000,8.0,8.0,426.428571,100.0,6.314775,2.0,5170.0,29.227271,1471.714286,1555326363,None,False
2,False,1.0,1.0,2.0,0.906580,15.0,44.0,96.000000,4.0,0.034722,2.0,629.0,14.828373,414.000000,1554971763,None,False
3,False,1.0,1.0,1.0,0.000000,0.0,0.0,40.000000,1.0,0.000012,1.0,4.0,3.058824,3.000000,1554969774,None,False
4,False,1.0,2.0,4.0,1.662506,14.0,10.0,67.000000,6.0,21.280683,4.0,3.0,2.058824,2.000000,1554967752,Feature Addition,False


In [439]:
#df = df.sample(frac=1)
label_col = 'contains_bug'
features_cols = ['fix', 'ns', 'nd', 'nf', 'entropy', 'la', 'ld', 'lt', 'ndev', 'age', 'nuc', 'exp', 'rexp', 'sexp', 'classification']
x = df[features_cols]
x['fix'] = x['fix'].astype('int')
df_classification = pd.get_dummies(x, columns=['classification'])
x = pd.concat([x, df_classification], axis='columns')
x = x.drop(['classification'], axis='columns')
x = x.values
y = df[label_col]
y = y.astype('category')
y = y.cat.codes
y = y.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [440]:
print(x[:3])
print(y[:3])

[[0.00000000e+00 7.00000000e+00 7.00000000e+00 7.00000000e+00
  2.64160417e+00 9.00000000e+00 9.00000000e+00 4.26428571e+02
  1.00000000e+02 9.25925926e-05 1.00000000e+00 5.17100000e+03
  3.02272705e+01 1.47271429e+03 0.00000000e+00 7.00000000e+00
  7.00000000e+00 7.00000000e+00 2.64160417e+00 9.00000000e+00
  9.00000000e+00 4.26428571e+02 1.00000000e+02 9.25925926e-05
  1.00000000e+00 5.17100000e+03 3.02272705e+01 1.47271429e+03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.00000000e+00 7.00000000e+00 7.00000000e+00
  2.75000000e+00 8.00000000e+00 8.00000000e+00 4.26428571e+02
  1.00000000e+02 6.31477513e+00 2.00000000e+00 5.17000000e+03
  2.92272705e+01 1.47171429e+03 0.00000000e+00 7.00000000e+00
  7.00000000e+00 7.00000000e+00 2.75000000e+00 8.00000000e+00
  8.00000000e+00 4.26428571e+02 1.00000000e+02 6.31477513e+00
  2.00000000e+00 5.17000000e+03 2.92272705e+01 1.47171429e+03
  0.00000000e+00 0.00000000e+00 0.000

In [0]:
val_index = int( len(x) * 0.8 )
test_index = int( len(x) * 0.9 )
x_train, y_train = x[:val_index], y[:val_index]
x_val, y_val = x[val_index:test_index], y[val_index:test_index]
x_test, y_test = x[test_index:], y[test_index:]

In [442]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[0.00000000e+00 7.00000000e+00 7.00000000e+00 7.00000000e+00
  2.64160417e+00 9.00000000e+00 9.00000000e+00 4.26428571e+02
  1.00000000e+02 9.25925926e-05 1.00000000e+00 5.17100000e+03
  3.02272705e+01 1.47271429e+03 0.00000000e+00 7.00000000e+00
  7.00000000e+00 7.00000000e+00 2.64160417e+00 9.00000000e+00
  9.00000000e+00 4.26428571e+02 1.00000000e+02 9.25925926e-05
  1.00000000e+00 5.17100000e+03 3.02272705e+01 1.47271429e+03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.00000000e+00 7.00000000e+00 7.00000000e+00
  2.75000000e+00 8.00000000e+00 8.00000000e+00 4.26428571e+02
  1.00000000e+02 6.31477513e+00 2.00000000e+00 5.17000000e+03
  2.92272705e+01 1.47171429e+03 0.00000000e+00 7.00000000e+00
  7.00000000e+00 7.00000000e+00 2.75000000e+00 8.00000000e+00
  8.00000000e+00 4.26428571e+02 1.00000000e+02 6.31477513e+00
  2.00000000e+00 5.17000000e+03 2.92272705e+01 1.47171429e+03
  0.00000000e+00 0.00000000e+00 0.000

In [0]:
mean_train = x_train.mean(axis=0)
std_train = x_train.std(axis=0)

In [444]:
print(mean_train)
print(std_train)

[2.90871477e-01 1.61432635e+00 2.60858547e+00 4.18908715e+00
 7.43390526e-01 5.00800621e+01 2.96331523e+01 8.11968929e+02
 2.32963538e+01 9.76994760e+01 2.02296354e+00 1.73095790e+03
 6.53773667e+01 8.28714959e+02 2.90871477e-01 1.61432635e+00
 2.60858547e+00 4.18908715e+00 7.43390526e-01 5.00800621e+01
 2.96331523e+01 8.11968929e+02 2.32963538e+01 9.76994760e+01
 2.02296354e+00 1.73095790e+03 6.53773667e+01 8.28714959e+02
 2.90871477e-01 1.47142488e-01 3.46521852e-02 4.71373157e-01
 1.51538660e-02 4.08068270e-02]
[4.54164354e-01 1.62736223e+00 6.93052365e+00 2.56616350e+01
 1.13236779e+00 5.86250473e+02 6.19852440e+02 1.15236538e+03
 2.84287684e+01 2.37802272e+02 7.67653474e+00 2.11471132e+03
 6.96820390e+01 1.34456679e+03 4.54164354e-01 1.62736223e+00
 6.93052365e+00 2.56616350e+01 1.13236779e+00 5.86250473e+02
 6.19852440e+02 1.15236538e+03 2.84287684e+01 2.37802272e+02
 7.67653474e+00 2.11471132e+03 6.96820390e+01 1.34456679e+03
 4.54164354e-01 3.54247902e-01 1.82897270e-01 4.99179

In [0]:
x_train = (x_train - mean_train) / std_train
x_val = (x_val - mean_train) / std_train
x_test = (x_test - mean_train) / std_train

In [446]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[-6.40454219e-01  3.30944983e+00  6.33633872e-01  1.09537559e-01
   1.67632253e+00 -7.00725440e-02 -3.32872003e-02 -3.34564335e-01
   2.69809951e+00 -4.10842935e-01 -1.33258504e-01  1.62671948e+00
  -5.04435529e-01  4.78964178e-01 -6.40454219e-01  3.30944983e+00
   6.33633872e-01  1.09537559e-01  1.67632253e+00 -7.00725440e-02
  -3.32872003e-02 -3.34564335e-01  2.69809951e+00 -4.10842935e-01
  -1.33258504e-01  1.62671948e+00 -5.04435529e-01  4.78964178e-01
  -6.40454219e-01 -4.15365869e-01 -1.89462561e-01  1.05899079e+00
  -1.24044505e-01 -2.06259221e-01]
 [-6.40454219e-01  3.30944983e+00  6.33633872e-01  1.09537559e-01
   1.77204746e+00 -7.17782996e-02 -3.49004875e-02 -3.34564335e-01
   2.69809951e+00 -3.84288594e-01 -2.99139370e-03  1.62624660e+00
  -5.18786429e-01  4.78220444e-01 -6.40454219e-01  3.30944983e+00
   6.33633872e-01  1.09537559e-01  1.77204746e+00 -7.17782996e-02
  -3.49004875e-02 -3.34564335e-01  2.69809951e+00 -3.84288594e-01
  -2.99139370e-03  1.62624660e+00 -5.1878

In [0]:
def create_sampler(y):
  _, counts = np.unique(y, return_counts=True)
  n_samples = len(y)
  class_weights = n_samples / counts
  weights = class_weights[y]
  fade = reversed(range(n_samples))
  fading_factor = 1 #0.999999
  fade = [fading_factor**x for x in (fade)]
  weights = weights * fade
  return data.WeightedRandomSampler(weights=weights, num_samples=n_samples, replacement=True)

In [0]:
sampler = create_sampler(y_train)

In [0]:
x_train, y_train = torch.from_numpy(x_train), torch.from_numpy(y_train)
x_val, y_val = torch.from_numpy(x_val), torch.from_numpy(y_val)
x_test, y_test = torch.from_numpy(x_test), torch.from_numpy(y_test)

In [0]:
train_dataset = data.TensorDataset(x_train, y_train)
val_dataset = data.TensorDataset(x_val, y_val)
test_dataset = data.TensorDataset(x_test, y_test)

train_dataloader = data.DataLoader(train_dataset, batch_size=512, sampler=sampler)
val_dataloader = data.DataLoader(val_dataset, batch_size=32)
test_dataloader = data.DataLoader(test_dataset, batch_size=32)

In [451]:
inputs, targets = next(iter(train_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(val_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(test_dataloader))
print(inputs[:3])
print(targets[:3])

tensor([[-0.6405, -0.3775, -0.2321, -0.1243, -0.6565, -0.0837, -0.0462, -0.6760,
         -0.6084, -0.4108, -0.1333, -0.6899,  0.1471, -0.5531, -0.6405, -0.3775,
         -0.2321, -0.1243, -0.6565, -0.0837, -0.0462, -0.6760, -0.6084, -0.4108,
         -0.1333, -0.6899,  0.1471, -0.5531, -0.6405, -0.4154, -0.1895,  1.0590,
         -0.1240, -0.2063],
        [-0.6405, -0.3775, -0.2321, -0.1243, -0.6565, -0.0837, -0.0462,  0.2022,
         -0.5029, -0.3536, -0.1333, -0.7490, -0.8580, -0.5308, -0.6405, -0.3775,
         -0.2321, -0.1243, -0.6565, -0.0837, -0.0462,  0.2022, -0.5029, -0.3536,
         -0.1333, -0.7490, -0.8580, -0.5308, -0.6405, -0.4154, -0.1895,  1.0590,
         -0.1240, -0.2063],
        [-0.6405, -0.3775, -0.2321, -0.1243, -0.6565, -0.0837, -0.0462, -0.6682,
         -0.6084, -0.2566, -0.1333, -0.8185, -0.9382, -0.6163, -0.6405, -0.3775,
         -0.2321, -0.1243, -0.6565, -0.0837, -0.0462, -0.6682, -0.6084, -0.2566,
         -0.1333, -0.8185, -0.9382, -0.6163, -0.6405,

In [0]:
class Classifier(nn.Module):
  def __init__(self, input_size, hidden_size, drop_prob, epoch=None, val_loss=None):
    super(Classifier, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.drop_prob = drop_prob
    self.epoch = epoch
    self.val_loss = val_loss
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fcout = nn.Linear(hidden_size, 1)
    self.dropout = nn.Dropout(drop_prob)
  
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = self.dropout(x)
    x = torch.sigmoid(self.fcout(x))
    return x

In [453]:
classifier = Classifier(input_size=x.shape[1], hidden_size=x.shape[1], drop_prob=0.5)
classifier

Classifier(
  (fc1): Linear(in_features=34, out_features=34, bias=True)
  (fcout): Linear(in_features=34, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [0]:
filename = 'classifier.cpt'
def save_classifier(classifier):
  checkpoint = {
      'input_size': classifier.input_size,
      'hidden_size': classifier.hidden_size,
      'drop_prob': classifier.drop_prob,
      'val_loss': classifier.val_loss,
      'epoch': classifier.epoch,
      'state_dict': classifier.state_dict()
  }
  with open(filename, 'wb') as f:
    torch.save(checkpoint, f)

def load_classifier():
  with open(filename, 'rb') as f:
    checkpoint = torch.load(f)
  classifier = Classifier(checkpoint['input_size'], checkpoint['hidden_size'], checkpoint['drop_prob'], checkpoint['epoch'], checkpoint['val_loss'])
  classifier.load_state_dict(checkpoint['state_dict'])
  return classifier

In [0]:
save_classifier(classifier)
classifier = load_classifier()

In [456]:
if torch.cuda.is_available():
  classifier = classifier.cuda()
  inputs = inputs.cuda()

classifier(inputs[:3].float())

tensor([[0.3300],
        [0.4429],
        [0.5332]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [0]:
criterion = nn.BCELoss()
optimizer = optim.Adam(params=classifier.parameters(), lr=0.003)

In [458]:
train_loss = 0
epochs = 200
for epoch in range(epochs):
  classifier.train()
  for inputs, targets in train_dataloader:
    if torch.cuda.is_available():
      inputs, targets = inputs.cuda(), targets.cuda()

    outputs = classifier(inputs.float())
    loss = criterion(outputs.squeeze(), targets.float())
    train_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  with torch.no_grad():
    classifier.eval()
    val_loss = 0
    for inputs, targets in val_dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

      outputs = classifier(inputs.float())
      loss = criterion(outputs.squeeze(), targets.float())
      val_loss += loss.item()

    train_loss = train_loss / len(train_dataloader)
    val_loss = val_loss / len(val_dataloader)
    print('Epoch: {}, Train loss: {}, Val loss: {}'.format(epoch, train_loss, val_loss))

    if classifier.val_loss is None or val_loss < classifier.val_loss:
      classifier.epoch = epoch
      classifier.val_loss = val_loss
      save_classifier(classifier)

classifier.epoch = epoch
classifier.val_loss = val_loss

Epoch: 0, Train loss: 0.6473907671476665, Val loss: 0.6584849655628204
Epoch: 1, Train loss: 0.6248877828636328, Val loss: 0.6206800616101215
Epoch: 2, Train loss: 0.6020582919197086, Val loss: 0.636688127329475
Epoch: 3, Train loss: 0.5995526235482065, Val loss: 0.6432105742002788
Epoch: 4, Train loss: 0.585846847403418, Val loss: 0.5957143189091432
Epoch: 5, Train loss: 0.583291485281063, Val loss: 0.6049797131042731
Epoch: 6, Train loss: 0.5790394736583628, Val loss: 0.6048770037136579
Epoch: 7, Train loss: 0.5751057444780949, Val loss: 0.5885274045561489
Epoch: 8, Train loss: 0.566900051421162, Val loss: 0.5876771348871683
Epoch: 9, Train loss: 0.5711412596280699, Val loss: 0.59112766894855
Epoch: 10, Train loss: 0.5683736429077892, Val loss: 0.5865606904814118
Epoch: 11, Train loss: 0.5668691763372341, Val loss: 0.582472391818699
Epoch: 12, Train loss: 0.5569955418222075, Val loss: 0.599867702706864
Epoch: 13, Train loss: 0.5546679332322011, Val loss: 0.5403762604845198
Epoch: 14,

In [0]:
def calculate_recalls(targets, predictions):
  n_classes = 2
  confusion_matrix, _, _ = np.histogram2d(targets.detach().cpu().numpy(), predictions.squeeze().detach().cpu().numpy(), bins=n_classes)
  return np.diag(confusion_matrix) / (np.sum(confusion_matrix, axis=1))

def gmean(classifier, dataloader):
  with torch.no_grad():
    classifier.eval()
    recalls = np.zeros((2))
    for inputs, targets in dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()
      
      outputs = classifier(inputs.float())
      predictions = torch.round(outputs).int()
      recalls += calculate_recalls(targets, predictions)

    recalls = recalls / len(dataloader)
    return mstats.gmean(recalls)    

In [0]:
def evaluate(classifier):
  if torch.cuda.is_available():
    classifier = classifier.cuda()
  train_accuracy = gmean(classifier, train_dataloader)
  test_accuracy = gmean(classifier, test_dataloader)
  print('Epoch: {}'.format(classifier.epoch))
  print('Train g-mean: {}'.format(train_accuracy.item()))
  print('Test g-mean: {}'.format(test_accuracy.item()))

In [461]:
print('Best classifier')
evaluate(load_classifier())

Best classifier
Epoch: 153
Train g-mean: 0.7685135808186105
Test g-mean: 0.735250675988632


In [462]:
print('Last classifier')
evaluate(classifier)

Last classifier
Epoch: 199
Train g-mean: 0.7726625191209947
Test g-mean: 0.7395796249173632
